In [1]:
!pip install pandas

In [2]:
import os
import scipy.io
import numpy as np
import pandas as pd

# Set data folder path
DATA_FOLDER = r'D:/NUS_TERM2_CA3/MAREA_dataset'

ACTIVITY_FOLDER = os.path.join(DATA_FOLDER, 'Activity Timings')
SUBJECT_FOLDER = os.path.join(DATA_FOLDER, 'Subject_Data_txt_format')


###########################################################################
# treadWalk       = indoor_time['indoorTime'][:,1:2]
# treadIncline    = indoor_time['indoorTime'][:,4:5]
# treadWalknRun   = indoor_time['indoorTime'][:,1:3]
# indoorWalk      = indoor_time['indoorTime'][:,6:7]
# indoorRun       = indoor_time['indoorTime'][:,6:8]
# outdoorWalk     = outdoor_time['outdoorTime'][:,1:2]
# outdoorWalknRun = outdoor_time['outdoorTime'][:,1:3]
###########################################################################

###########################################################################
# Subject numbers 1 to 11 are involved in Indoor Experiments
# Subject numbers 12 to 20 are involved in Outdoor Experiments
###########################################################################

###########################################################################
# There are five Indoor Activity labels:
# actIndex = 1 -> treadWalk
# actIndex = 2 -> treadIncline
# actIndex = 3 -> treadWalknRun
# actIndex = 4 -> indoorWalk
# actIndex = 5 -> indoorWalknRun

# There are two Outdoor Activity labels:
# actIndex = 1 -> outdoorWalk
# actIndex = 2 -> outdoorWalknRun
###########################################################################

###########################################################################
# There are four accelerometer positions (accPos) to choose from:
# accPos = 1 -> Left Foot
# accPos = 2 -> Right Foot
# accPos = 3 -> Waist
# accPos = 4 -> Wrist
###########################################################################

In [3]:
# Define Activity Labels
indoor_label = ['tread_flat_walk_start', 
                'tread_flat_walk_end',
                'tread_flat_run_end',
                'tread_slope_walk_start',
                'tread_slope_walk_end',
                'indoor_flat_walk_start',
                'indoor_flat_walk_end',
                'indoor_flat_run_end'
               ]

outdoor_label = ['outdoor_walk_start',
                 'outdoor_walk_end',
                 'outdoor_run_end']

indoor_time_df = pd.read_csv(os.path.join(ACTIVITY_FOLDER, 'Indoor Experiment Timings.txt')
                            , names= indoor_label)

outdoor_time_df = pd.read_csv(os.path.join(ACTIVITY_FOLDER, 'Outdoor Experiment Timings.txt')
                            , names=outdoor_label)

indoor_time_df["subject"] = ["Sub" + str(i) for i in range(1, 12)]
outdoor_time_df["subject"] = ["Sub" + str(j) for j in range(12, 21)]

print(indoor_time_df)
print(outdoor_time_df)

    tread_flat_walk_start  tread_flat_walk_end  tread_flat_run_end  \
0                       1                55931               85681   
1                       1                40911               84991   
2                       1                62261               83961   
3                       1                45781               84551   
4                       1                63971               85121   
5                       1                69381               84781   
6                       1                46331               83761   
7                       1                53801               84811   
8                       1                69131               84661   
9                       1                70041               84941   
10                      1                77641               84741   

    tread_slope_walk_start  tread_slope_walk_end  indoor_flat_walk_start  \
0                   102181                180281                  223681   
1      

In [3]:
pos_list = ['LF','RF','Waist','Wrist']
sub_list = ["Sub" + str(i) for i in range(1, 21)]
sub_list.remove('Sub4')


In [5]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
import numpy as np
import pywt

def alsbase(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

def denoise(signal_orig):
    coeffs_orig = pywt.wavedec(signal_orig, 'db4', level=2)
    coeffs_filter = coeffs_orig.copy()

    threshold = 0.8

    for i in range(1, len(coeffs_orig)):
        coeffs_filter[i] = pywt.threshold(coeffs_orig[i], threshold*max(coeffs_orig[i]))

    signal_denoised = pywt.waverec(coeffs_filter, 'db4')
    
    return signal_denoised

In [12]:

new_names = ['accX_LF', 'accY_LF', 'accZ_LF', 
            'accX_RF', 'accY_RF', 'accZ_RF', 
             'accX_Waist', 'accY_Waist', 'accZ_Waist', 
             'accX_Wrist', 'accY_Wrist', 'accZ_Wrist'            
            ]

sub_df = None

for sub in sub_list:
    lf_df = pd.read_csv(os.path.join(SUBJECT_FOLDER, sub + '_' + 'LF.txt'))
    rf_df = pd.read_csv(os.path.join(SUBJECT_FOLDER, sub + '_' + 'RF.txt'))
    waist_df = pd.read_csv(os.path.join(SUBJECT_FOLDER, sub + '_' + 'Waist.txt'))
    wrist_df = pd.read_csv(os.path.join(SUBJECT_FOLDER, sub + '_' + 'Wrist.txt'))
    sub_df = pd.concat([lf_df, rf_df, waist_df, wrist_df], axis=1)
    sub_df.columns = new_names
    print(sub_df.head())

    sub_df_new = sub_df.copy()
    sub_df_new = denoise(sub_df_new.values)
    sub_df_new.columns = new_names
    print(sub_df_new.head())

    for column in new_names:
        sub_df_new[column] = sub_df_new[column] - alsbase(sub_df_new[column], 10 ^ 5, 0.000005, niter=10)

    n = int(sub[3:])
    if n > 11:
        sub_row = outdoor_time_df[outdoor_time_df['subject'] == sub]
        tmp = sub_row.iloc[0]
        sub_df_new.loc[0:tmp['outdoor_walk_end'], 'label'] = 'outdoor_walk'
        sub_df_new.loc[tmp['outdoor_walk_end']: tmp['outdoor_run_end'], 'label'] = 'outdoor_run'
    else:
        sub_row = indoor_time_df[indoor_time_df['subject'] == sub]
        tmp = sub_row.iloc[0]
        sub_df_new.loc[0:tmp['tread_flat_walk_end'], 'label'] = 'tread_flat_walk'
        sub_df_new.loc[tmp['tread_flat_walk_end']: tmp['tread_flat_run_end'], 'label'] = 'tread_flat_run'
        sub_df_new.loc[tmp['tread_flat_run_end']: tmp['tread_slope_walk_start'], 'label'] = 'rest'
        sub_df_new.loc[tmp['tread_slope_walk_start']: tmp['tread_slope_walk_end'], 'label'] = 'tread_slope_walk'
        sub_df_new.loc[tmp['tread_slope_walk_end']: tmp['indoor_flat_walk_start'], 'label'] = 'rest'
        sub_df_new.loc[tmp['indoor_flat_walk_start']: tmp['indoor_flat_walk_end'], 'label'] = 'indoor_flat_walk'
        sub_df_new.loc[tmp['indoor_flat_walk_end']: tmp['indoor_flat_run_end'], 'label'] = 'indoor_flat_run'

    print(sub_df_new)
    sub_df_new.to_csv(sub + '_processed.csv')

   accX_LF  accY_LF  accZ_LF  accX_RF  accY_RF  accZ_RF  accX_Waist  \
0   -2.196  -11.765   -1.569    4.078   -6.902    2.196      -0.157   
1   -2.510  -10.353   -2.039    4.392   -8.000    1.255      -0.157   
2   -2.353  -10.824   -2.510    4.863   -8.314    0.941      -0.157   
3   -1.882  -10.667   -2.667    4.549   -9.412   -0.157      -0.157   
4   -1.412  -10.353   -2.824    3.451  -10.353   -0.784      -0.627   

   accY_Waist  accZ_Waist  accX_Wrist  accY_Wrist  accZ_Wrist  
0      -8.157       0.784       0.941     -11.451      -2.510  
1      -8.000       1.255       0.627     -10.510      -2.353  
2      -7.843       1.255       0.784     -10.196      -2.039  
3      -8.000       1.412       1.255     -10.039      -1.412  
4      -7.529       1.882       1.569     -10.196      -1.255  
(271681,)


ValueError: Length of values does not match length of index

In [4]:
import os

import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

DATA_FOLDER = r'D:/NUS_TERM2_CA3/MAREA_dataset'
PROCESSED_FOLDER = os.path.join(DATA_FOLDER, 'Processed_data')

sub_list = ["Sub" + str(i) for i in range(2, 21)]
sub_list.remove('Sub4')


full_df = pd.read_csv(os.path.join(PROCESSED_FOLDER, 'Sub1_processed.csv'))
full_df = full_df.drop(full_df.columns[[0]], axis=1)

print(full_df.shape)

for sub in sub_list:
    tmp_df = pd.read_csv(os.path.join(PROCESSED_FOLDER, sub + '_processed.csv'))
    tmp_df = tmp_df.drop(tmp_df.columns[[0]], axis=1)

    print('Loading ' + sub + '_processed.csv')
    print(tmp_df.shape)
    full_df = full_df.append(tmp_df, ignore_index = True)

print(full_df.shape)


(271681, 13)
Loading Sub2_processed.csv
(336391, 13)
Loading Sub3_processed.csv
(257861, 13)
Loading Sub5_processed.csv
(257021, 13)
Loading Sub6_processed.csv
(235981, 13)
Loading Sub7_processed.csv
(255761, 13)
Loading Sub8_processed.csv
(270311, 13)
Loading Sub9_processed.csv
(246761, 13)
Loading Sub10_processed.csv
(240741, 13)
Loading Sub11_processed.csv
(276041, 13)
Loading Sub12_processed.csv
(49651, 13)
Loading Sub13_processed.csv
(71011, 13)
Loading Sub14_processed.csv
(43581, 13)
Loading Sub15_processed.csv
(33140, 13)
Loading Sub16_processed.csv
(46651, 13)
Loading Sub17_processed.csv
(47261, 13)
Loading Sub18_processed.csv
(48801, 13)
Loading Sub19_processed.csv
(47141, 13)
Loading Sub20_processed.csv
(32201, 13)
(3067988, 13)


In [5]:
df_tread_flat_walk = full_df[full_df['label'] == 'tread_flat_walk']
print(df_tread_flat_walk.shape)

df_tread_flat_run = full_df[full_df['label'] == 'tread_flat_run']
print(df_tread_flat_run.shape)

df_tread_slope_walk = full_df[full_df['label'] == 'tread_slope_walk']
print(df_tread_slope_walk.shape)

df_indoor_flat_walk = full_df[full_df['label'] == 'indoor_flat_walk']
print(df_indoor_flat_walk.shape)

df_indoor_flat_run = full_df[full_df['label'] == 'indoor_flat_run']
print(df_indoor_flat_run.shape)

df_rest = full_df[full_df['label'] == 'rest']
print(df_rest.shape)

df_outdoor_walk = full_df[full_df['label'] == 'outdoor_walk']
print(df_outdoor_walk.shape)

df_outdoor_run = full_df[full_df['label'] == 'outdoor_run']
print(df_outdoor_run.shape)


(609400, 13)
(238050, 13)
(819700, 13)
(229200, 13)
(236600, 13)
(515600, 13)
(235839, 13)
(183599, 13)


In [6]:
window_size = 256
number_columns = 13

activity_to_num_mapping = {
    "rest":0,
    "tread_flat_walk":1,
    "tread_flat_run":2,
    "tread_slope_walk":3,
    "indoor_flat_walk":4,
    "indoor_flat_run":5,
    
    "outdoor_walk":6,
    "outdoor_run":7
}


def reshape_df(df, window_size, number_columns):
    n_drop = df.shape[0] % window_size
    n_samples = df.shape[0] // window_size
    df = df[:-n_drop]
    
    label = activity_to_num_mapping.get(df.iloc[0][12])
    label_series = pd.Series([label for _ in range(n_samples)])
    
    return df.values.reshape(n_samples, window_size, number_columns), label_series


In [7]:
df_tread_flat_walk_3d, ds_tread_flat_walk_label = reshape_df(df_tread_flat_walk, window_size, number_columns)
print(df_tread_flat_walk_3d.shape)
print(ds_tread_flat_walk_label.shape)

df_tread_flat_run_3d, ds_tread_flat_run_label = reshape_df(df_tread_flat_run, window_size, number_columns)
print(df_tread_flat_run_3d.shape)
print(ds_tread_flat_run_label.shape)

df_tread_slope_walk_3d, ds_tread_slope_walk_label = reshape_df(df_tread_slope_walk, window_size, number_columns)
print(df_tread_slope_walk_3d.shape)
df_indoor_flat_walk_3d, ds_indoor_flat_walk_label = reshape_df(df_indoor_flat_walk, window_size, number_columns)
print(df_indoor_flat_walk_3d.shape)
df_indoor_flat_run_3d, ds_indoor_flat_run_label = reshape_df(df_indoor_flat_run, window_size, number_columns)
print(df_indoor_flat_run_3d.shape)
df_outdoor_walk_3d, ds_outdoor_walk_label = reshape_df(df_outdoor_walk, window_size, number_columns)
print(df_outdoor_walk_3d.shape)
df_outdoor_run_3d, ds_outdoor_run_label = reshape_df(df_outdoor_run, window_size, number_columns)
print(df_outdoor_run_3d.shape)
df_rest_3d, ds_rest_label = reshape_df(df_rest, window_size, number_columns)
print(df_rest_3d.shape)

full_df_3d = np.vstack((df_tread_flat_walk_3d, df_tread_flat_run_3d,
                        df_tread_slope_walk_3d, df_indoor_flat_walk_3d,
                        df_indoor_flat_run_3d, df_outdoor_walk_3d,
                        df_outdoor_run_3d, df_rest_3d
                       ))

print(full_df_3d.shape)

full_ds_label = ds_tread_flat_walk_label.append(ds_tread_flat_run_label)
full_ds_label= full_ds_label.append(ds_tread_slope_walk_label)
full_ds_label= full_ds_label.append(ds_indoor_flat_walk_label)
full_ds_label= full_ds_label.append(ds_indoor_flat_run_label)
full_ds_label= full_ds_label.append(ds_outdoor_walk_label)
full_ds_label= full_ds_label.append(ds_outdoor_run_label)
full_ds_label= full_ds_label.append(ds_rest_label)

print(full_ds_label.shape)


(2380, 256, 13)
(2380,)
(929, 256, 13)
(929,)
(3201, 256, 13)
(895, 256, 13)
(924, 256, 13)
(921, 256, 13)
(717, 256, 13)
(2014, 256, 13)
(11981, 256, 13)
(11981,)


In [8]:
df_array = full_df_3d[:,:,:-1]
print(df_array.shape)

from keras.utils.np_utils import to_categorical

y_cat = to_categorical(full_ds_label, num_classes=8)
print(y_cat)
print(y_cat.shape)

(11981, 256, 12)


Using TensorFlow backend.
c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\guofe\workspace\nus_is_pr\

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(11981, 8)


c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\guofe\workspace\nus_is_pr\venv\

In [9]:
y = y_cat
X = df_array

padding_number = 19 #12000 - 11981 

df_padding_X = df_array[0:19,:,:]
df_padding_y = y_cat[0:19,:]

print(df_padding_X.shape)
print(df_padding_y.shape)

X = np.vstack((X, df_padding_X))
y = np.vstack((y, df_padding_y))

print(X.shape)
print(y.shape)

(19, 256, 12)
(19, 8)
(12000, 256, 12)
(12000, 8)


In [32]:
# y = full_df['label']
# print(y.shape)
# print(type(y))
# X = full_df.drop('label', axis=1)

In [11]:
!pip install sklearn

  Using cached https://files.pythonhosted.org/packages/1e/7a/dbb3be0ce9bd5c8b7e3d87328e79063f8b263b2b1bfa4774cb1147bfcd3f/sklearn-0.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/d6/9e/6a42486ffa64711fb868e5d4a9167153417e7414c3d8d3e0d627cf391e1e/scikit_learn-0.21.3-cp37-cp37m-win_amd64.whl
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


In [10]:
from sklearn.model_selection import train_test_split, cross_val_score


#https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 / 4, random_state=5)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# print(X_train.describe())
# print(y_train.describe())

(9000, 256, 12)
(9000, 8)
(3000, 256, 12)
(3000, 8)


In [16]:
!pip install keras

  Using cached https://files.pythonhosted.org/packages/bc/3f/4f733cd0b1b675f34beb290d465a65e0f06b492c00b111d1b75125062de1/PyYAML-5.1.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl


In [18]:
!pip install tensorflow-gpu

  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
Processing c:\users\guofe\appdata\local\pip\cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6\termcolor-1.1.0-cp37-none-any.whl
Processing c:\users\guofe\appdata\local\pip\cache\wheels\d7\de\2e\efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd\wrapt-1.11.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/00/83/b4a77d044e78ad1a45610eb88f745be2fd2c6d658f9798a15e384b7d57c9/wheel-0.33.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d0/33/376510eb8d6246f3c30545f416b2263eee461e40940c2a4413c711bdf62d/google_pasta-0.1.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c0/4e/fd492e91abdc2d2fcb70ef45

In [11]:
# ------------------------------
# https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()

batch_size = 300

# Recurrent layer
model.add(LSTM(64, batch_input_shape=(batch_size, 256, 12), return_sequences=False, dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(128, activation='relu'))

# Dropout for regularization
# model.add(Dropout(0.5))

# Output layer
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

STEPS = X_train.shape[0] // 20
# VALID_STEPS = validation_generator.n // 20

history = model.fit(X_train, y_train, epochs=200, batch_size=batch_size, validation_split=0.2, verbose=1)


W1026 13:27:11.112962 15212 deprecation_wrapper.py:119] From c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1026 13:27:11.165019 15212 deprecation_wrapper.py:119] From c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1026 13:27:11.166964 15212 deprecation_wrapper.py:119] From c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1026 13:27:11.228002 15212 deprecation_wrapper.py:119] From c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

Train on 7200 samples, validate on 1800 samples
Epoch 1/200
7200/7200 [==============================] - 14s 2ms/step - loss: 1.9150 - acc: 0.3010 - val_loss: 1.6595 - val_acc: 0.3867
Epoch 2/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.5174 - acc: 0.4247 - val_loss: 1.3555 - val_acc: 0.4672
Epoch 3/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.3086 - acc: 0.4800 - val_loss: 1.1547 - val_acc: 0.5167
Epoch 4/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.1544 - acc: 0.5257 - val_loss: 1.0578 - val_acc: 0.5556
Epoch 5/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.0768 - acc: 0.5557 - val_loss: 1.0392 - val_acc: 0.5589
Epoch 6/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.0392 - acc: 0.5654 - val_loss: 0.9918 - val_acc: 0.5844
Epoch 7/200
7200/7200 [==============================] - 12s 2ms/step - loss: 1.0178 - acc: 0.5790 - val_loss: 0.9735 - val_acc: 0.5967


7200/7200 [==============================] - 12s 2ms/step - loss: 0.6129 - acc: 0.7707 - val_loss: 0.5809 - val_acc: 0.7772
Epoch 61/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.6405 - acc: 0.7567 - val_loss: 0.5756 - val_acc: 0.7794
Epoch 62/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.6110 - acc: 0.7700 - val_loss: 0.5972 - val_acc: 0.7728
Epoch 63/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.6078 - acc: 0.7696 - val_loss: 0.6076 - val_acc: 0.7594
Epoch 64/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.6015 - acc: 0.7771 - val_loss: 0.5706 - val_acc: 0.7778
Epoch 65/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5846 - acc: 0.7786 - val_loss: 0.5770 - val_acc: 0.7778
Epoch 66/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5821 - acc: 0.7874 - val_loss: 0.5624 - val_acc: 0.7822
Epoch 67/200
7200/7200 [==============================

Epoch 120/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4575 - acc: 0.8340 - val_loss: 0.4343 - val_acc: 0.8561
Epoch 121/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4656 - acc: 0.8318 - val_loss: 0.4432 - val_acc: 0.8417
Epoch 122/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4537 - acc: 0.8318 - val_loss: 0.4460 - val_acc: 0.8500
Epoch 123/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4490 - acc: 0.8350 - val_loss: 0.4453 - val_acc: 0.8461
Epoch 124/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4460 - acc: 0.8360 - val_loss: 0.4360 - val_acc: 0.8517
Epoch 125/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4509 - acc: 0.8388 - val_loss: 0.4172 - val_acc: 0.8544
Epoch 126/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.4474 - acc: 0.8342 - val_loss: 0.4404 - val_acc: 0.8422
Epoch 127/200
7200/7200 [=========

7200/7200 [==============================] - 12s 2ms/step - loss: 0.3653 - acc: 0.8675 - val_loss: 0.3894 - val_acc: 0.8728
Epoch 180/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3607 - acc: 0.8711 - val_loss: 0.3756 - val_acc: 0.8761
Epoch 181/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3729 - acc: 0.8660 - val_loss: 0.3793 - val_acc: 0.8733
Epoch 182/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3700 - acc: 0.8622 - val_loss: 0.3656 - val_acc: 0.8767
Epoch 183/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3562 - acc: 0.8671 - val_loss: 0.3661 - val_acc: 0.8756
Epoch 184/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3680 - acc: 0.8646 - val_loss: 0.3970 - val_acc: 0.8700
Epoch 185/200
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3703 - acc: 0.8650 - val_loss: 0.3713 - val_acc: 0.8744
Epoch 186/200
7200/7200 [=======================

In [12]:
print(model.summary())

score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (300, 64)                 19712     
_________________________________________________________________
dense_1 (Dense)              (300, 128)                8320      
_________________________________________________________________
dense_2 (Dense)              (300, 8)                  1032      
Total params: 29,064
Trainable params: 29,064
Non-trainable params: 0
_________________________________________________________________
None
3000/3000 [==============================] - 2s 827us/step
Test Loss: 0.3608592689037323
Test accuracy: 0.8799999952316284


In [13]:
model_json = model.to_json()
with open("ca3_model_v2.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights('ca3_model_v2.h5')
print("Saved model to disk")

Saved model to disk


In [ ]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('ca3_model_v2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ca3_model_v2.h5")
print("Loaded model from disk")

# Compile the model
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
print(model.summary())

score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])